## Test Scrape Notebook

In [20]:
## Import library as establish paths
from pypdf import PdfReader
import translators as ts
import pandas as pd
import nltk
from docx import Document


headername = r"Output File"

filename = 'source_file'

source_pdf_input_path = r'./input/' + filename + r'.pdf'
source_text_output_path = r'./output/' + filename + '.txt'
trans_table_output_path = r'./output/' + filename + '_translation.csv'
trans_text_output_path = r'./output/' + filename + '_translated.txt'
trans_doc_output_path = r'./output/' + filename + '_translated.docx'


## Data Prep

In [ ]:
reader = PdfReader(source_pdf_input_path)
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [ ]:
print(number_of_pages - 856)
# Pulling pages 775 to 830

In [ ]:
# Trying to extract the range into a text file
select_range = list(range(775,831,1))

long_text = ''

for p in select_range:
    page = reader.pages[p]
    text = page.extract_text()
    long_text = long_text + text
    
print(len(long_text))




In [ ]:
# Write out to raw text
# with open(source_text_output_path, 'w', encoding="utf-8") as f:
#     f.write(long_text)

## Translation

In [2]:
# read raw text
with open(source_text_output_path, 'r', encoding="utf-8") as f:
    long_text = f.read()
print(len(long_text))


105090


In [3]:
## step 1: convert raw text into sentences in a df
from nltk.tokenize import sent_tokenize

sentences_list = sent_tokenize(long_text)

print(len(sentences_list))



379


In [4]:
## establish a dataframe to store input and output
trans_df = pd.DataFrame()
trans_df['raw_nl'] = sentences_list

print(trans_df.head())

In [5]:
# import the library
import googletrans
from googletrans import Translator

In [9]:
# create a translator object
translator = Translator()

# use translate method to translate a string - by default, the destination language is english
translated = translator.translate(trans_df['raw_nl'][0],
                                  src='nl',
                                  dest= 'en')

# the translate method returns an object
print(translated.text)

THIRTEENTH MEETING .


In [11]:
## df style

trans_df['trans_en'] = trans_df['raw_nl'].apply(lambda x: translator.translate(x,
                                  src='nl',
                                  dest= 'en').text)
print(trans_df.head())

In [13]:
## write out the table
trans_df.to_csv(trans_table_output_path)

In [16]:
## write out the translated text
long_text_translated = ''.join(trans_df["trans_en"])
# Write out to raw text
with open(trans_text_output_path, 'w', encoding="utf-8") as f:
    f.write(long_text_translated)


In [22]:
## as word doc
document = Document()
document.add_heading(headername, level=1)
document.add_paragraph(long_text_translated)

document.save(trans_doc_output_path)